## Testing of model Transitive V1

This notebook aims to use cross validation to test the performance of mode Transitive V1.

Transitive V1 is a simple model that instead of predicting matches, aims to predict the probability of a player winning any given point. Then, with dynamic programming we convert the point probabilities to a match win probability. This model only considers two different types of points, service and return points.

We estimate a player's point % win rate against another player by using common opponents. Based on the performance of each player against these common opponents, we estimate the relative serve and return win % between the two plyers.

2024 data is reserved for the validation set.

In [1]:
import numpy as np
import pandas as pd

from atp_forecaster.models.transitive_v1 import TransitiveV1
from atp_forecaster.data.clean import get_cleaned_atp_matches

matches = get_cleaned_atp_matches()
matches[matches['tourney_date'] >= 20240101].head()


,surface,draw_size,tourney_level,tourney_date,id_a,name_a,hand_a,ht_a,age_a,id_b,...,2ndWon_b,SvGms_b,bpSaved_b,bpFaced_b,rank_a,rank_points_a,rank_b,rank_points_b,result,order
88725,Hard,32,A,20240101,209992,Juncheng Shang,L,180.0,18.9,126207,...,11.0,10.0,6.0,8.0,183.0,335.0,16.0,2310.0,1,88725
88726,Hard,32,A,20240101,209950,Arthur Fils,R,185.0,19.5,126094,...,15.0,13.0,7.0,10.0,36.0,1158.0,5.0,4805.0,0,88726
88727,Hard,32,A,20240101,200325,Emil Ruusuvuori,R,188.0,24.7,124116,...,22.0,15.0,9.0,12.0,69.0,771.0,43.0,1048.0,1,88727
88728,Hard,32,A,20240101,209992,Juncheng Shang,L,180.0,18.9,126094,...,19.0,13.0,5.0,8.0,183.0,335.0,5.0,4805.0,0,88728
88729,Hard,32,A,20240101,126094,Andrey Rublev,R,188.0,26.1,200325,...,12.0,10.0,5.0,7.0,5.0,4805.0,69.0,771.0,1,88729


In [2]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score

def test_transitive(model, start_date: int, end_date: int):
    """
    Test the performance of the Transitive V1 model over a given date range.
    """
    # Get all matches between start_date and end_date
    train = matches[matches['tourney_date'] < start_date]
    test = matches[(matches['tourney_date'] >= start_date) & (matches['tourney_date'] < end_date)]

    model.fit(train)

    y_pred = []
    y_true = []

    # Predict the outcome of each match in the test set
    for (_, row) in test.iterrows():
        if (len(model.find_common_opponents(row['id_a'], row['id_b'])) < 20):
            continue

        prob = model.predict_match(row['id_a'], row['id_b'])

        y_pred.append(prob)
        y_true.append(row['result'])

        print(row['name_a'], row['name_b'], prob)

        # update model with the match
        model.add_match(row)
    
    y_pred = np.array(y_pred, dtype=float)
    y_true = np.array(y_true, dtype=int)

    loss = log_loss(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)
    acc = accuracy_score(y_true, (y_pred > 0.5).astype(int))

    return loss, auc, acc

In [3]:
model = TransitiveV1(
    look_back=720,
    base_spw=0.6
)

loss, auc, acc = test_transitive(model, 20230101, 20230401)
print("Log Loss: ", loss)
print("AUC: ", auc)
print("Accuracy: ", acc)

Sebastian Korda Andy Murray 0.6265577982514196
Yoshihito Nishioka Holger Rune 0.5305942609345194
Marcos Giron Richard Gasquet 0.5763675460104118
Roberto Bautista Agut Andrey Rublev 0.4335501789609936
Alexei Popyrin Felix Auger Aliassime 0.5343198566587557
Marcos Giron Alexei Popyrin 0.4477220192870192
Yoshihito Nishioka Mackenzie Mcdonald 0.5838874369506596
Roberto Bautista Agut Sebastian Korda 0.45112627341669187
Jannik Sinner Thanasi Kokkinakis 0.28817261872574595
Daniil Medvedev Miomir Kecmanovic 0.5460574950856237
Daniil Medvedev Lorenzo Sonego 0.6148360292769663
Sebastian Baez Pedro Martinez 0.4619433085634044
Aslan Karatsev Pedro Martinez 0.585258954831563
Matteo Berrettini Thiago Monteiro 0.6917965625421453
Sebastian Korda Jannik Sinner 0.5280959178778248
Alexander Bublik Hubert Hurkacz 0.43314415679331375
David Goffin Grigor Dimitrov 0.46701904678603584
David Goffin Stefanos Tsitsipas 0.49720704371423496
Cameron Norrie Taylor Fritz 0.4599366156083085
Matteo Berrettini Hubert Hu